In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import warnings
warnings.simplefilter("ignore")

In [2]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
select houses.name, houses.state, destinations.name ->> 'fr' as destination, 
CASE WHEN houses.created_at::date < '1/1/2020' then '1/1/2020' else houses.created_at::date end as created_at,
'12/31/2023'::date as max_date
from houses
left join destinations on destinations.id = houses.destination_id
where houses.state in ('published', 'hidden', 'warning')
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [3]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
df['max_date'] = pd.to_datetime(df['max_date'])

In [4]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'destination': r['destination'], 'state': r['state'], 'possible_days': pd.date_range(start=r['created_at'], end=r['max_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'D'
# group by period
df['period'] = df['possible_days'].dt.to_period(period_type)
df = df.groupby(['name', 'destination', 'state', 'period']).agg({'possible_days': 'count'}).reset_index()
df

,name,destination,state,period,possible_days
0,Villa Linda,Punta Cana,published,2022-09-08,1
1,Villa Linda,Punta Cana,published,2022-09-09,1
2,Villa Linda,Punta Cana,published,2022-09-10,1
3,Villa Linda,Punta Cana,published,2022-09-11,1
4,Villa Linda,Punta Cana,published,2022-09-12,1
...,...,...,...,...,...
2605094,Zaira Estate,Paros,published,2023-12-27,1
2605095,Zaira Estate,Paros,published,2023-12-28,1
2605096,Zaira Estate,Paros,published,2023-12-29,1
2605097,Zaira Estate,Paros,published,2023-12-30,1


In [5]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name','destination', 'state'], columns='period', values='possible_days').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,destination,state,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,...,2023-12-22,2023-12-23,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31
0,Villa Linda,Punta Cana,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,A Mandria,Région de Porto-Vecchio,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Alpaca,Méribel,published,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Alpine Myth,Sankt Anton,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Alykes Estate,Corfou,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,Villa Échappée,Sainte-Maxime,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2861,Villas Doppia,Presqu'ile Saint-Tropez,hidden,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2862,Villas Iberis,Gassin,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2863,Wood Penthouse,Saint-Moritz,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name', 'destination', 'state'], var_name='period', value_name='possible_days')
unpivoted_df

,name,destination,state,period,possible_days
0,Villa Linda,Punta Cana,published,2020-01-01,0.0
1,A Mandria,Région de Porto-Vecchio,published,2020-01-01,0.0
2,Alpaca,Méribel,published,2020-01-01,1.0
3,Alpine Myth,Sankt Anton,published,2020-01-01,0.0
4,Alykes Estate,Corfou,published,2020-01-01,0.0
...,...,...,...,...,...
4185760,Villa Échappée,Sainte-Maxime,published,2023-12-31,1.0
4185761,Villas Doppia,Presqu'ile Saint-Tropez,hidden,2023-12-31,1.0
4185762,Villas Iberis,Gassin,published,2023-12-31,1.0
4185763,Wood Penthouse,Saint-Moritz,published,2023-12-31,1.0


In [7]:
df_hub = pd.read_csv(r"C:\Users\Benjamin\LC-Clean\Business_dashboard_lab\mapping_hub.csv", sep=';')
unpivoted_df = unpivoted_df.merge(df_hub, on='destination', how='left')

In [8]:
unpivoted_df_final = unpivoted_df[["cluster", "destination", "name", "state", "period", "possible_days"]]
unpivoted_df_final

,cluster,destination,name,state,period,possible_days
0,ROW,Punta Cana,Villa Linda,published,2020-01-01,0.0
1,Corse,Région de Porto-Vecchio,A Mandria,published,2020-01-01,0.0
2,Alpes LC,Méribel,Alpaca,published,2020-01-01,1.0
3,Alpes Autrichienne,Sankt Anton,Alpine Myth,published,2020-01-01,0.0
4,Grèce,Corfou,Alykes Estate,published,2020-01-01,0.0
...,...,...,...,...,...,...
4185760,French Riviera,Sainte-Maxime,Villa Échappée,published,2023-12-31,1.0
4185761,French Riviera,Presqu'ile Saint-Tropez,Villas Doppia,hidden,2023-12-31,1.0
4185762,French Riviera,Gassin,Villas Iberis,published,2023-12-31,1.0
4185763,Alpes LC,Saint-Moritz,Wood Penthouse,published,2023-12-31,1.0


In [9]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
SELECT houses.name,
case when check_in_date::date < '1/1/2020' then '1/1/2020' else check_in_date::date end as check_in_date,
case when check_out_date::date >= '1/1/2024' then '12/31/2023' else check_out_date::date-1 end as check_out_date
FROM bookings
left join houses on houses.id = bookings.house_id
left join destinations on destinations.id = houses.destination_id
where check_out_date > '1/1/2020' and check_in_date < '1/1/2024' and houses.name != 'House test' and (booker_type = 'Contract') and houses.state in ('published', 'hidden', 'warning')
order by check_in_date asc
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [10]:
df

,name,check_in_date,check_out_date
0,Villa Palombe,2020-01-01,2020-01-03
1,Villa Agaves,2020-01-01,2020-01-02
2,Chalet Aline,2020-01-01,2020-01-04
3,Villa Toudra,2020-01-01,2020-01-03
4,Chalet Ilulissat,2020-01-01,2020-01-03
...,...,...,...
5187,Penthouse Phoenix 901 Le George,2023-12-23,2023-12-29
5188,Appartement Doron Lodge,2023-12-23,2023-12-29
5189,Villa Avana,2023-12-25,2023-12-31
5190,Villa Alham,2023-12-27,2023-12-31


In [11]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['check_in_date'] = pd.to_datetime(df['check_in_date'])
df['check_out_date'] = pd.to_datetime(df['check_out_date'])

In [12]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'booking_days_lc': pd.date_range(start=r['check_in_date'], end=r['check_out_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'D'
# group by period
df['period'] = df['booking_days_lc'].dt.to_period(period_type)
df = df.groupby(['name', 'period']).agg({'booking_days_lc': 'count'}).reset_index()
df

,name,period,booking_days_lc
0,A Mandria,2022-06-15,1
1,A Mandria,2022-06-16,1
2,A Mandria,2022-06-17,1
3,A Mandria,2022-06-18,1
4,A Mandria,2022-08-21,1
...,...,...,...
47261,Zaira Estate,2023-08-14,1
47262,Zaira Estate,2023-08-15,1
47263,Zaira Estate,2023-08-16,1
47264,Zaira Estate,2023-08-17,1


In [13]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name'], columns='period', values='booking_days_lc').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,...,2023-11-03,2023-12-23,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31
0,A Mandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Appartement Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Appartement Arche Lodge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Appartement Arly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Appartement Beaurepaire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Villa les Arbousiers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1232,Villa les Cigales,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1233,Villa les Roches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,Villa Échappée,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name'], var_name='period', value_name='booking_days_lc')
unpivoted_df

,name,period,booking_days_lc
0,A Mandria,2020-01-01,0.0
1,Appartement Alaska,2020-01-01,0.0
2,Appartement Arche Lodge,2020-01-01,0.0
3,Appartement Arly,2020-01-01,0.0
4,Appartement Beaurepaire,2020-01-01,0.0
...,...,...,...
1737811,Villa les Arbousiers,2023-12-31,0.0
1737812,Villa les Cigales,2023-12-31,0.0
1737813,Villa les Roches,2023-12-31,0.0
1737814,Villa Échappée,2023-12-31,0.0


In [15]:
unpivoted_df_final = unpivoted_df_final.merge(unpivoted_df, on=['name', 'period'], how='left')

In [16]:
unpivoted_df_final = unpivoted_df_final.fillna(0)

In [17]:
unpivoted_df_final

,cluster,destination,name,state,period,possible_days,booking_days_lc
0,ROW,Punta Cana,Villa Linda,published,2020-01-01,0.0,0.0
1,Corse,Région de Porto-Vecchio,A Mandria,published,2020-01-01,0.0,0.0
2,Alpes LC,Méribel,Alpaca,published,2020-01-01,1.0,0.0
3,Alpes Autrichienne,Sankt Anton,Alpine Myth,published,2020-01-01,0.0,0.0
4,Grèce,Corfou,Alykes Estate,published,2020-01-01,0.0,0.0
...,...,...,...,...,...,...,...
4185760,French Riviera,Sainte-Maxime,Villa Échappée,published,2023-12-31,1.0,0.0
4185761,French Riviera,Presqu'ile Saint-Tropez,Villas Doppia,hidden,2023-12-31,1.0,0.0
4185762,French Riviera,Gassin,Villas Iberis,published,2023-12-31,1.0,0.0
4185763,Alpes LC,Saint-Moritz,Wood Penthouse,published,2023-12-31,1.0,0.0


In [18]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
SELECT houses.name,
case when check_in_date::date < '1/1/2020' then '1/1/2020' else check_in_date::date end as check_in_date,
case when check_out_date::date >= '1/1/2024' then '12/31/2023' else check_out_date::date-1 end as check_out_date
FROM bookings
left join houses on houses.id = bookings.house_id
left join destinations on destinations.id = houses.destination_id
where check_out_date > '1/1/2020' and check_in_date < '1/1/2024' and houses.name != 'House test' and (booker_type != 'Contract' or type != 'Bookings::SalesOption') and houses.state in ('published', 'hidden', 'warning')
order by check_in_date asc
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [19]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['check_in_date'] = pd.to_datetime(df['check_in_date'])
df['check_out_date'] = pd.to_datetime(df['check_out_date'])

In [20]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'booking_days_all': pd.date_range(start=r['check_in_date'], end=r['check_out_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'D'
# group by period
df['period'] = df['booking_days_all'].dt.to_period(period_type)
df = df.groupby(['name', 'period']).agg({'booking_days_all': 'count'}).reset_index()
df

,name,period,booking_days_all
0,Villa Linda,2022-10-06,1
1,Villa Linda,2022-10-07,1
2,Villa Linda,2022-10-08,1
3,Villa Linda,2022-10-09,1
4,Villa Linda,2022-11-04,1
...,...,...,...
737202,Zaira Estate,2023-08-14,1
737203,Zaira Estate,2023-08-15,1
737204,Zaira Estate,2023-08-16,1
737205,Zaira Estate,2023-08-17,1


In [21]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name'], columns='period', values='booking_days_all').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,...,2023-12-22,2023-12-23,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31
0,Villa Linda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A Mandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alpaca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alpine Myth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alykes Estate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,Villa les Roches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2689,Villa Échappée,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2690,Villas Doppia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2691,Villas Iberis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [22]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name'], var_name='period', value_name='booking_days_all')
unpivoted_df

,name,period,booking_days_all
0,Villa Linda,2020-01-01,0.0
1,A Mandria,2020-01-01,0.0
2,Alpaca,2020-01-01,0.0
3,Alpine Myth,2020-01-01,0.0
4,Alykes Estate,2020-01-01,0.0
...,...,...,...
3934468,Villa les Roches,2023-12-31,0.0
3934469,Villa Échappée,2023-12-31,0.0
3934470,Villas Doppia,2023-12-31,0.0
3934471,Villas Iberis,2023-12-31,0.0


In [23]:
unpivoted_df_final = unpivoted_df_final.merge(unpivoted_df, on=['name', 'period'], how='left')

In [24]:
unpivoted_df_final = unpivoted_df_final.fillna(0)

In [25]:
unpivoted_df_final_group = unpivoted_df_final.groupby(['period']).agg({'possible_days':'sum', 'booking_days_lc':'sum', 'booking_days_all':'sum'}).reset_index()
unpivoted_df_final_group['occupancy_rate_lc'] = unpivoted_df_final_group['booking_days_lc']/unpivoted_df_final_group['possible_days']*100
unpivoted_df_final_group['occupancy_rate_all'] = unpivoted_df_final_group['booking_days_all']/unpivoted_df_final_group['possible_days']*100

In [26]:
unpivoted_df_final_group = unpivoted_df_final_group.round(2)

In [27]:
unpivoted_df_final_group.to_csv('occupancy_rate.csv')

In [28]:
unpivoted_df_final_group.to_clipboard()